In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from sqlalchemy import create_engine

In [2]:
with open('sample-data.json', mode = 'r', encoding = 'utf-8') as file:
    data = json.load(file)

In [3]:
#načtení dat
vstup = pd.DataFrame(data)

In [4]:
#tvorba prázdného dataframu
vystup = pd.DataFrame(columns = ['name', 'cpu_use', 'memory_use', 'created_at', 'status', 'IP'])

In [11]:
#zápis dat do df
vystup.name = vstup.name

for i,e in enumerate(vstup.state):
    try:
        vystup.memory_use[i] = vstup.state[i]['memory']['usage']
    except:
        pass

for i, e in enumerate(vstup.created_at):
    vystup.created_at[i] = pd.to_datetime(vstup.created_at[i]).tz_convert('UTC')

vystup.status = vstup.status

for i,e in enumerate(vstup.state):
    try:
        network = pd.DataFrame.from_dict(vstup.state[i]['network'])
    except:
        pass
    ips = []
    for j in network.loc['addresses']:
        for k in j:
            ips.append(k['address'])
    vystup.IP[i] = ips

In [12]:
#převedu IP a datum na string
vystup.IP = vystup.IP.apply(str)
vystup.created_at = vystup.created_at.apply(str)

In [15]:
#zapíšu do databáze
connection = sqlite3.connect("Teska.db")
engine = create_engine('sqlite://', echo=False)
vystup.to_sql('vystup', con=connection, index = False)